In [92]:
import json
with open('ebird_config.json') as f:
    data = json.load(f)
    recent_species_observations = data['recent_species_observations']
    ebird_token = data['ebird_token']
    gmap_api_key = data['gmap_api_key']
# recent_species_observations

In [93]:
import requests
headers = {}
headers['X-eBirdApiToken'] = ebird_token
response = requests.get(recent_species_observations, headers=headers)
res_json = response.json()

In [94]:
# res_json

In [95]:
import datetime as dt
import collections as col
DATE_FORMAT = '%Y-%m-%d %H:%M'
Sighting = col.namedtuple('Sighting', ['lat', 'lng', 'date'])
# 0-th is today, 1-st is yesterday, etc
sightings = list()
for sighting in res_json:
    sightings.append(Sighting(lat=sighting['lat'], lng=sighting['lng'],
                              date=dt.datetime.strptime(sighting['obsDt'], DATE_FORMAT)))
today = dt.date.today()
sightings_last_seven_days = list() # today is included
for i in range(0, 7):
    sightings_last_seven_days.append(
        list(filter(lambda s: s.date.date() == today - dt.timedelta(days=i), sightings)))
#sightings_last_seven_days

In [112]:
layers = list()
# the more red the color, the closer to today
colors = list(['red', 'rgb(255, 165, 0)', 'yellow', 'green', 'blue', 'rgb(75, 0, 130)', 'rgb(238, 130, 238)'])
for day_sightings in sightings_last_seven_days:
    layers.append(list(map(lambda s: (s.lat, s.lng), day_sightings)))

In [137]:
import gmaps
gmaps.configure(api_key=gmap_api_key)
fig = gmaps.figure()
def add_layer(lat_lng, color, scale):
    layer = gmaps.symbol_layer(lat_lng, fill_color=color, stroke_color=color, scale=scale)
    fig.add_layer(layer)
# not counting today, used to further limit the 7 days to a smaller number of days, if desired
#days_back = int(input('Number of days to show, 0 for today only '))
days_back = 3
days_back = max(0, days_back)
# add the layers backwards so today is on top, but it does not seem to work,
# looks like there is no support for setting zIndex on a symbol (or symbols of a layer)
for i in range(min(days_back, 6), -1, -1):
    add_layer(layers[i], colors[i], (4 - i) if (days_back <= 3) else int((8-i)/2))
#fig

In [138]:
import matplotlib.pyplot as plt